In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler
#Import scikit-learn metrics module for accuracy calculation
from sklearn import metrics
#from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler
from sklearn.ensemble import IsolationForest
from sklearn import preprocessing
import calendar
import time

print("Importing data")
#test_data = pd.read_csv("C:/Users/silaks/Desktop/Simo/SEB Big Data Challange/Python/test_data_1.csv")
#data = pd.read_csv("C:/Users/silaks/Desktop/Simo/SEB Big Data Challange/Python/full.csv")
data = pd.read_csv("C:/Users/silaks/OneDrive - dematic.com/Desktop/python/full.csv")
local_test_data = pd.read_csv("C:/Users/silaks/OneDrive - dematic.com/Desktop/python/test_data_1.csv")

print("Converting categoricals to dummies (one hot encoding)")
# There are around 20 unique categories
# As data is not too big, we can afford to create additional columns
# With more data and time it would make sense to combine some categories based on similiarity
one_hot_encoded_data = pd.get_dummies(data)
one_hot_encoded_test_data = pd.get_dummies(local_test_data)

print("Save full and test IDs")
# This will be required later to distinguish local test and train sets
one_hot_encoded_test_data["test_id"] = one_hot_encoded_test_data["id"]
one_hot_encoded_data["test_id"] = 0
one_hot_encoded_data["full_id"] = one_hot_encoded_data["id"]
one_hot_encoded_test_data["full_id"] = 0
ohe_data = one_hot_encoded_data
ohe_test_data = one_hot_encoded_test_data

print("Remove columns")
# These columns do not exist in the test set
ohe_data = ohe_data.drop('loan_purpose_UFVCU1VC', axis = 1)
ohe_data = ohe_data.drop('loan_purpose_renewable_energy', axis = 1)
# Could just ignore these columns in model later
ohe_data = ohe_data.drop('id', axis = 1)
ohe_test_data = ohe_test_data.drop('id', axis = 1)
## ALL Y VALUES OF TEST SET ARE MISSING - THESE ARE UNKNOWN
ohe_test_data = ohe_test_data.drop('y', axis = 1)
## ALL Y VALUES OF TEST SET ARE MISSING - THESE ARE UNKNOWN

print("Dealing with NAs")
###############################################################
######### Remove NA rows
ohe_data.drop(ohe_data[ohe_data["max_open_credit"].isnull()].index, inplace = True) 
ohe_data.drop(ohe_data[ohe_data["bankruptcies"].isnull()].index, inplace = True) 
ohe_data.drop(ohe_data[ohe_data["years_current_job"].isnull()].index, inplace = True) 
###############################################################
############ Replace NAs
# If there was no delinquent ever - a highest value is placed
# People who had delinquent very long time ago are more likely to be "good" loans
ohe_data["months_since_last_delinquent"] = ohe_data["months_since_last_delinquent"].fillna(value = 300)
ohe_test_data["months_since_last_delinquent"] = ohe_test_data["months_since_last_delinquent"].fillna(value = 300)
# Alternative would be to introduce a categorical variable

print("Forming label y vector")
labels = np.array(ohe_data['y'])
# Remove the labels from the features
# axis 1 refers to the columns
ohe_data = ohe_data.drop('y', axis = 1)


#############################
#### Try that gave good results with unknown test set - combining train and test data to impute missing values in both
print("Pre-combined data shape:",ohe_data.shape)
ohe_data_combined_imputed_income = ohe_data
ohe_data_combined_imputed_income = ohe_data_combined_imputed_income.append(ohe_test_data)
ohe_data_combined_imputed_income = ohe_data_combined_imputed_income.sample(frac = 1)
ohe_data_combined_imputed_income = ohe_data_combined_imputed_income.reset_index()
ohe_data_combined_imputed_income = ohe_data_combined_imputed_income.drop('index', axis = 1)
print("Post-combined data shape:",ohe_data_combined_imputed_income.shape)




Importing data
Converting to dummies
Save full and test IDs
Remove columns
Dealing with NAs
Forming label y vector
Pre-combined data shape: (9559809, 35)


C:\Users\silaks\AppData\Local\Temp\ipykernel_2944\2578939567.py:64: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ohe_data_combined_imputed_income = ohe_data_combined_imputed_income.append(ohe_test_data)


Post-combined data shape: (9569809, 35)


term
2
['short' 'long']

credit_score
4
['very_good' 'good' nan 'fair']

loan_purpose
16
['other' 'debt_consolidation' 'home_improvements' 'take_a_trip'
 'medical_bills' 'educational_expenses' 'major_purchase' 'small_business'
 'business_loan' 'buy_a_car' 'moving' 'buy_house' 'vacation' 'wedding'
 'renewable_energy' 'UFVCU1VC']

home_ownership
3
['rent' 'mortgage' 'own']



In [ ]:
########################################################
####### IMPUTING YEARLY INCOME - imputed in train and test sets
print("IMPUTING YEARLY INCOME")
imputer = KNNImputer(n_neighbors=2000)
n = 25000  #chunk row size
list_df = [ohe_data_combined_imputed_income[i:i+n] for i in range(0,ohe_data_combined_imputed_income.shape[0],n)]

ts = time.gmtime()
print("Imputing start time:",time.strftime("%Y-%m-%d %H:%M:%S", ts))
    
for i in range(1,len(list_df)):   
    if i % 20 == 0:
        ts = time.gmtime()
        print("Imputing set (",i*n," rows):",i,"   ",time.strftime("%Y-%m-%d %H:%M:%S", ts))
    list_df[i] = pd.DataFrame(imputer.fit_transform(list_df[i]),columns = list_df[i].columns)
    
ohe_data_full = pd.concat(list_df)
ohe_data_full["yearly_income"] = ohe_data_full["yearly_income"].fillna(ohe_data_full["yearly_income"].mean())


########################################################
####### LOAN AMOUNT 9999999
# This has some sort of meaning - leaving it as a new flag column
def loan_amount_category (row):
    if row['amount_current_loan'] == 99999999 :
        return 1
    else:
        return 0

ohe_data_full["loan_flag"] = ohe_data_full.apply(lambda row: loan_amount_category(row), axis=1)
ohe_data_full.loc[ohe_data_full.amount_current_loan == 99999999, 'amount_current_loan'] = None



########################################################
####### IMPUTING LOAN AMOUNT
# After setting the new flag - imputing the value to something realistic which will not mess the model
imputer = KNNImputer(n_neighbors=2000)
n = 25000  #chunk row size
list_df = [ohe_data_full[i:i+n] for i in range(0,ohe_data_full.shape[0],n)]

ts = time.gmtime()
print("Imputing start time:",time.strftime("%Y-%m-%d %H:%M:%S", ts))
    
for i in range(1,len(list_df)):   
    if i % 20 == 0:
        ts = time.gmtime()
        print("Imputing set (",i*n," rows):",i,"   ",time.strftime("%Y-%m-%d %H:%M:%S", ts))
    list_df[i] = pd.DataFrame(imputer.fit_transform(list_df[i]),columns = list_df[i].columns)
    
ohe_data_full = pd.concat(list_df)
ohe_data_full["amount_current_loan"] = ohe_data_full["amount_current_loan"].fillna(ohe_data_full["amount_current_loan"].mean())


In [20]:
########################################################
####### SEPARATING DATA FULL AND TEST (after imputation)
ohe_test_data_fixed = ohe_data_full[ohe_data_full["test_id"] != 0]
ohe_test_data_fixed = ohe_test_data_fixed.sort_values(by=['test_id'])

ohe_data_fixed = ohe_data_full[ohe_data_full["full_id"] != 0]
ohe_data_fixed = ohe_data_fixed.sort_values(by=['full_id'])

ohe_data_fixed = ohe_data_fixed.drop('test_id', axis = 1)
ohe_test_data_fixed = ohe_test_data_fixed.drop('test_id', axis = 1)
ohe_data_fixed = ohe_data_fixed.drop('full_id', axis = 1)
ohe_test_data_fixed = ohe_test_data_fixed.drop('full_id', axis = 1)


########################################################
####### RESETTING INDEXES
ohe_data_scaled_imputed = ohe_data_fixed
ohe_test_data_scaled_imputed = ohe_test_data_fixed
ohe_test_data_scaled_imputed = ohe_test_data_scaled_imputed.reset_index()
ohe_test_data_scaled_imputed = ohe_test_data_scaled_imputed.drop('index', axis = 1)
ohe_data_scaled_imputed = ohe_data_scaled_imputed.reset_index()
ohe_data_scaled_imputed = ohe_data_scaled_imputed.drop('index', axis = 1)



########################################################
####### PREPARING FEATURES
feature_list = list(ohe_data_scaled_imputed.columns)
print(feature_list)

#print(ohe_data.isna().sum())
features = ohe_data_scaled_imputed
#features = features.fillna(value = -1)
features = np.array(features)

features_final_test = ohe_test_data_scaled_imputed
#features_final_test = features_final_test.fillna(value = -1)
features_final_test = np.array(features_final_test)


# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
#train_features = features
#train_labels = labels

print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

print('FINAL Testing Features Shape:', features_final_test.shape)


###----------- THIS PART BELOW WAS REPEATED A MILLION TIMES, CHANGING PARAMETERS BASED ON RESULTS -------------------

print("Model train start time:",time.strftime("%Y-%m-%d %H:%M:%S", ts))
rf1 = RandomForestClassifier(n_estimators = 300, 
                            max_depth = 36, 
                            min_samples_split = 800,
                            n_jobs = -1,
                            random_state = 42,
                            max_features = 32,
                            oob_score = True,
                            class_weight = {0:4,1:1},
                            min_samples_leaf = 300, 
                            verbose = 1)

# Train the model on training data
rf1.fit(train_features, train_labels)
print("[28] AUC:",metrics.roc_auc_score(test_labels, rf1.predict_proba(test_features)[:,1]))
y_pred = rf1.predict_proba(features_final_test)[:,1]
pd.DataFrame(y_pred).to_csv("test_results_28_prob_prediction.csv")
#24 - 0.79...
#25 - 0.871377
#26 - 0.8384 - 50 trees
#27 - 0.8390 - 100 trees
print("Model end time:",time.strftime("%Y-%m-%d %H:%M:%S", ts))



print("Model train start time:",time.strftime("%Y-%m-%d %H:%M:%S", ts))
rf2 = RandomForestClassifier(n_estimators = 400, 
                            max_depth = 40, 
                            min_samples_split = 1000,
                            n_jobs = -1,
                            random_state = 42,
                            max_features = 34,
                            oob_score = True,
                            class_weight = {0:4,1:1},
                            min_samples_leaf = 500, 
                            verbose = 1)

# Train the model on training data
rf2.fit(train_features, train_labels)
print("[29] AUC:",metrics.roc_auc_score(test_labels, rf2.predict_proba(test_features)[:,1]))
y_pred = rf2.predict_proba(features_final_test)[:,1]
pd.DataFrame(y_pred).to_csv("test_results_29_prob_prediction.csv")
print("Model end time:",time.strftime("%Y-%m-%d %H:%M:%S", ts))



print("Model train start time:",time.strftime("%Y-%m-%d %H:%M:%S", ts))
rf3 = RandomForestClassifier(n_estimators = 400, 
                            max_depth = 46, 
                            min_samples_split = 3000,
                            n_jobs = -1,
                            random_state = 42,
                            max_features = 34,
                            oob_score = True,
                            class_weight = {0:3,1:1},
                            min_samples_leaf = 1000, 
                            verbose = 1)

# Train the model on training data
rf3.fit(train_features, train_labels)
print("[30] AUC:",metrics.roc_auc_score(test_labels, rf3.predict_proba(test_features)[:,1]))
y_pred = rf3.predict_proba(features_final_test)[:,1]
pd.DataFrame(y_pred).to_csv("test_results_30_prob_prediction.csv")
print("Model end time:",time.strftime("%Y-%m-%d %H:%M:%S", ts))

NameError: name 'ohe_data_full' is not defined

In [ ]:
########################################################
####### FEATURE IMPORTANCE
importances = rf.feature_importances_
std = np.std([tree.feature_importances_ for tree in rf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Feature ranking:")
for f in range(train_features.shape[1]):
    print("%d. %s (%f)" % (f + 1, feature_list[f], importances[indices[f]]))

# Plot the impurity-based feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(train_features.shape[1]), importances[indices],
        color="r", yerr=std[indices], align="center")
plt.xticks(range(train_features.shape[1]), indices)
plt.xlim([-1, train_features.shape[1]])
plt.show()